In [9]:
cd Z:\###_PhD1_###\RebeccaBeveridge - P27 CdkCyclin Fdc1\CdkCyclin\ION_MOBILITY\CCSDs

Z:\###_PhD1_###\RebeccaBeveridge - P27 CdkCyclin Fdc1\CdkCyclin\ION_MOBILITY\CCSDs


In [10]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource,Column 
from bokeh.models import HoverTool
from bokeh.models.widgets import Panel, Tabs
import seaborn as sns
import random
output_notebook()
%matplotlib inline
from IPython.display import HTML

def randomColorGenerator():
    colorGen = ''.join([random.choice('0123456789ABCDEF') for x in range(6)])
    return ''.join(["#",colorGen])

Loading BokehJS ...

In [11]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### Interactive tools:
#### <font color='green'>Hover</font>  - hover over any data point and it will bring detailed information about each curve (index point, CCS value, CCSD intensity and charge state)
#### <font color='green'>Zoom</font> - drag on the plot to zoom on specific region of interest
#### <font color='green'>Legend</font> - click on legend marker to select/deselect datasets 

In [12]:
# To do...
# Add possibility to normalize data to 1
# Add possibility to fill under the curve

In [13]:
def dataLoad(filename=None):
    """ Function to load data from CSV file """
    # Import data
    data = pd.read_csv(filename, delimiter=',')
    ccsData = data.CCS
    intGlobal = data['global']
    intCharge = data.loc[:, data.columns.str.startswith('z')]
    dataOut = [ccsData, intGlobal, intCharge]
    return dataOut

def prepareBokehTool():
    """ Function to prepare a tool for use with the Bokeh plot"""
    hover = HoverTool(tooltips=[("Point #:", "$index"),
                               ("CCS (Å²):", "$x{int} Å²"),
                               ("Normalized intensity:", "$y{0.00}"),
                               ("Charge state:", "@xlabel")])
    TOOLS = [hover,"box_zoom,resize,pan,wheel_zoom,reset,save"]
    return hover, TOOLS

def makeBokehFigure(dataIn=None, tool=None, plotWidth=800, plotHeight=750, xRange=(500,3000),
                   yRange=(0,1.05), title=None, activeTool="box_zoom", tabbed=False):
    
    t = figure(tools=tool, plot_width=plotWidth, plot_height=plotHeight, x_range=xRange
              , y_range=yRange, title=title, active_drag=activeTool)
    i = 0 
    dataSource = []
    ccsData, intGlobal, intCharge = dataOut
    for key in intCharge:
        legendKey = ''.join([key[1::],"+"])
        source = ColumnDataSource(data=dict(x=ccsData, y=intCharge[key], xlabel=([legendKey]*(len(ccsData)))))
        dataSource.append(source)
        t.line(x='x', y='y', line_color=randomColorGenerator(), line_width=4, alpha=0.7, legend=legendKey,
              source=dataSource[i]) 
        i=i+1
    source = ColumnDataSource(data=dict(x=ccsData, y=intGlobal, xlabel=(['Global']*(len(ccsData)))))
    dataSource.append(source)
    t.line(x='x', y='y', line_color='#FF0080', line_width=4, alpha=0.7, legend="Global", source=dataSource[i])       
    
    t.title.text_font_size = "16pt"
    t.xaxis.major_label_text_font_size = "12pt"
    t.xaxis.axis_label = 'Collision Cross Section (Å²)'
    t.xaxis.axis_label_text_font_size = "16pt"
    t.yaxis.major_label_text_font_size = "12pt"
    t.yaxis.axis_label = 'Normalized Intensity'
    t.yaxis.axis_label_text_font_size = "16pt"
    t.legend.click_policy="hide"
    if tabbed == True:
        tab = Panel(child=t, title=title)
        return tab
    else:
        show(t)
        return None

def makeBokehFigureFilled(dataIn=None, tool=None, plotWidth=800, plotHeight=750, xRange=(500,3000),
                   yRange=(0,1.05), title=None, activeTool="box_zoom", tabbed=False):
    hover, tool = prepareBokehTool()
    t = figure(tools=tool, plot_width=plotWidth, plot_height=plotHeight, x_range=xRange
              , y_range=yRange, title=title, active_drag=activeTool)
    i = 0 
    dataSource = []
    ccsData, intGlobal, intCharge = dataOut
    for key in intCharge:
        legendKey = ''.join([key[1::],"+"])
        source = ColumnDataSource(data=dict(x=ccsData, y=intCharge[key], xlabel=([legendKey]*(len(ccsData)))))
        dataSource.append(source)
        dataColor = randomColorGenerator()
        test = t.patch(x='x', y='y', line_color=dataColor, line_width=4, alpha=0.4, legend=legendKey,
              source=dataSource[i], color=dataColor) 
        i=i+1
        hover.renderers.append(test)
    source = ColumnDataSource(data=dict(x=ccsData, y=intGlobal, xlabel=(['Global']*(len(ccsData)))))
    dataSource.append(source)
    test = t.line(x='x', y='y', line_color='#FF0080', line_width=4, alpha=0.7, legend="Global", source=dataSource[i])       
    hover.renderers.append(test)
    t.title.text_font_size = "16pt"
    t.xaxis.major_label_text_font_size = "12pt"
    t.xaxis.axis_label = 'Collision Cross Section (Å²)'
    t.xaxis.axis_label_text_font_size = "16pt"
    t.yaxis.major_label_text_font_size = "12pt"
    t.yaxis.axis_label = 'Normalized Intensity'
    t.yaxis.axis_label_text_font_size = "16pt"
    t.legend.click_policy="hide"
    if tabbed == True:
        tab = Panel(child=t, title=title)
        return tab
    else:
        show(t)
        return None

# <font color='blue'>Tabbed View</font> 

In [14]:
plotHeight = 650
filename = 'p27c.csv'
title = 'p27-C-WT'
dataOut = dataLoad(filename=filename)
hover, tool = prepareBokehTool()
tab1 = makeBokehFigure(dataIn=dataOut, tool=tool, plotWidth=800, plotHeight=plotHeight, xRange=(500,3000),
                        yRange=(0,1.05), title=title, activeTool="box_zoom", tabbed=True)

filename = 'p27kid.csv'
title = 'p27-KID'
dataOut = dataLoad(filename=filename)
hover, tool = prepareBokehTool()
tab2 = makeBokehFigure(dataIn=dataOut, tool=tool, plotWidth=800, plotHeight=plotHeight, xRange=(500,3000),
                        yRange=(0,1.05), title=title, activeTool="box_zoom", tabbed=True)

filename = 'p27fl.csv'
title = 'p27-FL'
dataOut = dataLoad(filename=filename)
hover, tool = prepareBokehTool()
tab3 = makeBokehFigure(dataIn=dataOut, tool=tool, plotWidth=800, plotHeight=plotHeight, xRange=(500,8000),
                yRange=(0,1.05), title=title, activeTool="box_zoom", tabbed=True)

filename = 'cdkcyclin.csv'
title = 'Cdk:Cyclin'
dataOut = dataLoad(filename=filename)
hover, tool = prepareBokehTool()
tab4 = makeBokehFigure(dataIn=dataOut, tool=tool, plotWidth=800, plotHeight=plotHeight, xRange=(500,8000),
                yRange=(0,1.05), title=title, activeTool="box_zoom", tabbed=True)

filename = 'cdkcyclin_p27kid.csv'
title = 'Cdk:Cyclin:p27-KID'
dataOut = dataLoad(filename=filename)
hover, tool = prepareBokehTool()
tab5 = makeBokehFigure(dataIn=dataOut, tool=tool, plotWidth=800, plotHeight=plotHeight, xRange=(500,8000),
                yRange=(0,1.05), title=title, activeTool="box_zoom", tabbed=True)

filename = 'cdkcyclin_p27fl.csv'
title = 'Cdk:Cyclin:p27-FL'
dataOut = dataLoad(filename=filename)
hover, tool = prepareBokehTool()
tab6 = makeBokehFigure(dataIn=dataOut, tool=tool, plotWidth=800, plotHeight=plotHeight, xRange=(500,8000),
                yRange=(0,1.05), title=title, activeTool="box_zoom", tabbed=True)

tabs = Tabs(tabs=[ tab1, tab2, tab3, tab4, tab5, tab6 ])
show(tabs)

# p27-C-WT

In [15]:
filename = 'p27c.csv'
title = 'p27-C-WT'
dataOut = dataLoad(filename=filename)
hover, tool = prepareBokehTool()
makeBokehFigure(dataIn=dataOut, tool=tool, plotWidth=800, plotHeight=750, xRange=(500,3000),
                yRange=(0,1.05), title=title, activeTool="box_zoom")

# p27-KID

In [16]:
filename = 'p27kid.csv'
title = 'p27-KID'
dataOut = dataLoad(filename=filename)
hover, tool = prepareBokehTool()
makeBokehFigure(dataIn=dataOut, tool=tool, plotWidth=800, plotHeight=750, xRange=(500,3000),
                yRange=(0,1.05), title=title, activeTool="box_zoom")

# p27-FL

In [17]:
filename = 'p27fl.csv'
title = 'p27-FL'
dataOut = dataLoad(filename=filename)
hover, tool = prepareBokehTool()
makeBokehFigure(dataIn=dataOut, tool=tool, plotWidth=800, plotHeight=750, xRange=(500,8000),
                yRange=(0,1.05), title=title, activeTool="box_zoom")

# Cdk:Cyclin

In [18]:
filename = 'cdkcyclin.csv'
title = 'Cdk:Cyclin'
dataOut = dataLoad(filename=filename)
hover, tool = prepareBokehTool()
makeBokehFigure(dataIn=dataOut, tool=tool, plotWidth=800, plotHeight=750, xRange=(500,8000),
                yRange=(0,1.05), title=title, activeTool="box_zoom")

# Cdk:Cyclin:p27-KID

In [19]:
filename = 'cdkcyclin_p27kid.csv'
title = 'Cdk:Cyclin:p27-KID'
dataOut = dataLoad(filename=filename)
hover, tool = prepareBokehTool()
makeBokehFigure(dataIn=dataOut, tool=tool, plotWidth=800, plotHeight=750, xRange=(500,8000),
                yRange=(0,1.05), title=title, activeTool="box_zoom")

# Cdk:Cyclin:p27-FL

In [20]:
filename = 'cdkcyclin_p27fl.csv'
title = 'Cdk:Cyclin:p27-FL'
dataOut = dataLoad(filename=filename)
hover, tool = prepareBokehTool()
makeBokehFigure(dataIn=dataOut, tool=tool, plotWidth=800, plotHeight=750, xRange=(500,8000),
                yRange=(0,1.05), title=title, activeTool="box_zoom")